# 공통사항

## Import Library

In [1]:
# visualization
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'/usr/share/fonts/truetype/nanum/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns

# utils
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import warnings;warnings.filterwarnings('ignore')
import gdown

# Model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
import lightgbm as lgb

import eli5
from eli5.sklearn import PermutationImportance

## Data Load

In [2]:
train_path = '../../data/train.csv'
test_path  = '../../data/test.csv'
dt = pd.read_csv(train_path)
dt_test = pd.read_csv(test_path)

## Data Preprocessing

In [3]:
# train/test 구분을 위한 칼럼을 하나 만들어 줍니다.
# 데이터를 동일하게 처리해주기 유용함.
dt['is_test'] = 0
dt_test['is_test'] = 1
df = pd.concat([dt, dt_test])     # 하나의 데이터로 만들어줍니다.

In [4]:
# 필요 없어 보이는 columns 제거
drop_col = ['부번', '계약일', 'k-전화번호', 'k-팩스번호', 'k-관리방식', 'k-복도유형', 'k-시행사', 'k-사용검사일-사용승인일', 'k-홈페이지', 'k-등록일자', 'k-수정일자', '고용보험관리번호', '경비비관리형태', '세대전기계약방법', '청소비관리형태', '기타/의무/임대/임의=1/2/3/4', '단지승인일', '사용허가여부', '관리비 업로드', '단지신청일', 'k-관리비부과면적', '주차대수', '건축면적', '해제사유발생일', '단지소개기존clob', 'k-135㎡초과', '중개사소재지', '등기신청일자']
df.drop(drop_col, axis=1, inplace=True)

In [5]:
df['구'] = df['시군구'].apply(lambda x:x.split()[1])
df['동'] = df['시군구'].apply(lambda x:x.split()[2])

omg = ['용산구', '강남구', '서초구', '송파구', '성동구', '종로구']
is_omg = []
for x in df['구'].tolist():
    if x in omg:
        is_omg.append(1)
    else:
        is_omg.append(0)
df['개비싸'] = is_omg
# 이렇게 말고 '동' 을 분류하지 않아도 될듯       
df.loc[~df['구'].isin(omg), '동'] = 'Unknown' 

del df['시군구']

In [6]:
# 위 처럼 아무 의미도 갖지 않는 칼럼은 결측치와 같은 역할을 하므로, np.nan으로 채워 결측치로 인식되도록 합니다.
df['거래유형'] = df['거래유형'].replace('-', np.nan)

In [7]:
# 본번, 부번의 경우 float로 되어있지만 범주형 변수의 의미를 가지므로 object(string) 형태로 바꾸어주고 아래 작업을 진행하겠습니다.
df['본번'] = df['본번'].astype('str')

In [8]:
df['계약년'] = df['계약년월'].astype('str').map(lambda x : x[:4])
df['계약월'] = df['계약년월'].astype('str').map(lambda x : x[4:])

In [9]:
# 'time_col' 데이터를 문자열 형태로 변환
df['계약년월'] = df['계약년월'].astype(str)
# 문자열 형태 데이터를 datetime 형태로 변환
df['계약년월'] = pd.to_datetime(df['계약년월'], format='%Y%m')
# 변환 확인
print(df['계약년월'].head())

0   2017-12-01
1   2017-12-01
2   2017-12-01
3   2018-01-01
4   2018-01-01
Name: 계약년월, dtype: datetime64[ns]


In [10]:
# 먼저, 연속형 변수와 범주형 변수를 위 info에 따라 분리해주겠습니다.
continuous_columns = []
categorical_columns = []

for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        continuous_columns.append(column)
    else:
        categorical_columns.append(column)

print("연속형 변수:", continuous_columns)
print("범주형 변수:", categorical_columns)

연속형 변수: ['전용면적(㎡)', '층', '건축년도', 'k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '좌표X', '좌표Y', 'target', 'is_test', '개비싸']
범주형 변수: ['번지', '본번', '아파트명', '계약년월', '도로명', '거래유형', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)', 'k-난방방식', 'k-건설사(시공사)', '구', '동', '계약년', '계약월']


In [11]:
# 수치형 데이터를 어떻게 채워야 될지 모르겠음 -> 걍 빼.
# 좌표X, 좌표Y 를 리니어로 채우는건 혼동을 줄 수 있는 데이터임
df.drop(columns=['k-전체동수', 'k-전체세대수', 'k-연면적', 'k-주거전용면적', 'k-전용면적별세대현황(60㎡이하)', 'k-전용면적별세대현황(60㎡~85㎡이하)', 'k-85㎡~135㎡이하', '좌표X', '좌표Y'], inplace=True)

In [12]:
# 범주형 변수에 대한 보간
df[categorical_columns] = df[categorical_columns].fillna('NULL')

## Feature engineering

In [13]:
df_train = df.loc[df['is_test']==0, :]
df_test = df.loc[df['is_test']==1, :]

df_train.drop(['is_test'], axis=1, inplace=True)
df_test.drop(['is_test'], axis=1, inplace=True)
print(df_train.shape, df_test.shape)

(1118822, 19) (9272, 19)


In [14]:
# dt_test의 target은 일단 0으로 임의로 채워주도록 하겠습니다.
df_test['target'] = 0

In [15]:
# 파생변수 제작으로 추가된 변수들이 존재하기에, 다시한번 연속형과 범주형 칼럼을 분리해주겠습니다.
continuous_columns_v2 = []
categorical_columns_v2 = []

for column in df_train.columns:
    if column == '계약년월':
        continue
    if pd.api.types.is_numeric_dtype(df_train[column]):
        continuous_columns_v2.append(column)
    else:
        categorical_columns_v2.append(column)

print("연속형 변수:", continuous_columns_v2)
print("범주형 변수:", categorical_columns_v2)

연속형 변수: ['전용면적(㎡)', '층', '건축년도', 'target', '개비싸']
범주형 변수: ['번지', '본번', '아파트명', '도로명', '거래유형', 'k-단지분류(아파트,주상복합등등)', 'k-세대타입(분양형태)', 'k-난방방식', 'k-건설사(시공사)', '구', '동', '계약년', '계약월']


In [16]:
# 아래에서 범주형 변수들을 대상으로 레이블인코딩을 진행해 주겠습니다.

# 각 변수에 대한 LabelEncoder를 저장할 딕셔너리
label_encoders = {}

# Implement Label Encoding
for col in tqdm( categorical_columns_v2 ):
    lbl = LabelEncoder()

    # Label-Encoding을 fit
    lbl.fit( df_train[col].astype(str) )
    df_train[col] = lbl.transform(df_train[col].astype(str))
    label_encoders[col] = lbl           # 나중에 후처리를 위해 레이블인코더를 저장해주겠습니다.

    # Test 데이터에만 존재하는 새로 출현한 데이터를 신규 클래스로 추가해줍니다.
    for label in np.unique(df_test[col]):
      if label not in lbl.classes_: # unseen label 데이터인 경우
        lbl.classes_ = np.append(lbl.classes_, label) # 미처리 시 ValueError발생하니 주의하세요!

    df_test[col] = lbl.transform(df_test[col].astype(str))

100%|██████████| 13/13 [00:03<00:00,  3.89it/s]


In [17]:
df_train.head()        # 레이블인코딩이 된 모습입니다.

,번지,본번,아파트명,전용면적(㎡),계약년월,층,건축년도,도로명,거래유형,"k-단지분류(아파트,주상복합등등)",k-세대타입(분양형태),k-난방방식,k-건설사(시공사),target,구,동,개비싸,계약년,계약월
0,4732,1149,328,79.97,2017-12-01,3,1987,6176,0,3,2,1,241,124000.0,0,3,1,10,11
1,4732,1149,328,79.97,2017-12-01,4,1987,6176,0,3,2,1,241,123500.0,0,3,1,10,11
2,4732,1149,328,54.98,2017-12-01,5,1987,6176,0,3,2,1,241,91500.0,0,3,1,10,11
3,4732,1149,328,79.97,2018-01-01,4,1987,6176,0,3,2,1,241,130000.0,0,3,1,11,0
4,4732,1149,328,79.97,2018-01-01,2,1987,6176,0,3,2,1,241,117000.0,0,3,1,11,0


# Model Training

https://gmnam.tistory.com/230

In [18]:
assert df_train.shape[1] == df_test.shape[1]          # train/test dataset의 shape이 같은지 확인해주겠습니다.

In [19]:
def preprocess_feature_name(feature_name):
  """특수 문자를 제거하고 소문자로 변환합니다."""
  feature_name = feature_name.replace("-", "_")
  feature_name = feature_name.replace(",", "_")
  feature_name = feature_name.replace(".", "_")
  feature_name = feature_name.replace("(", "_")
  feature_name = feature_name.replace(")", "_")
  feature_name = feature_name.lower()
  return feature_name

def apply_preprocessed_feature_names(df_train):
  """데이터 프레임의 feature 이름을 수정합니다."""
  df_train.columns = [preprocess_feature_name(feature) for feature in df_train.columns]
  return df_train

# 데이터 프레임에 적용
df_train = apply_preprocessed_feature_names(df_train.copy())
df_test = apply_preprocessed_feature_names(df_test.copy())

# 확인
print(df_train.columns)

Index(['번지', '본번', '아파트명', '전용면적_㎡_', '계약년월', '층', '건축년도', '도로명', '거래유형',
       'k_단지분류_아파트_주상복합등등_', 'k_세대타입_분양형태_', 'k_난방방식', 'k_건설사_시공사_', 'target',
       '구', '동', '개비싸', '계약년', '계약월'],
      dtype='object')


In [20]:
df_train.head()

,번지,본번,아파트명,전용면적_㎡_,계약년월,층,건축년도,도로명,거래유형,k_단지분류_아파트_주상복합등등_,k_세대타입_분양형태_,k_난방방식,k_건설사_시공사_,target,구,동,개비싸,계약년,계약월
0,4732,1149,328,79.97,2017-12-01,3,1987,6176,0,3,2,1,241,124000.0,0,3,1,10,11
1,4732,1149,328,79.97,2017-12-01,4,1987,6176,0,3,2,1,241,123500.0,0,3,1,10,11
2,4732,1149,328,54.98,2017-12-01,5,1987,6176,0,3,2,1,241,91500.0,0,3,1,10,11
3,4732,1149,328,79.97,2018-01-01,4,1987,6176,0,3,2,1,241,130000.0,0,3,1,11,0
4,4732,1149,328,79.97,2018-01-01,2,1987,6176,0,3,2,1,241,117000.0,0,3,1,11,0


In [36]:
# Holdout - Random
y_train = df_train['target']
X_train = df_train.drop(['target', '계약년월'], axis=1)

holdout_random_X_train, holdout_random_X_val, holdout_random_y_train, holdout_random_y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=2023)

In [37]:
gbm = lgb.LGBMRegressor(n_estimators=1000)
gbm.fit(holdout_random_X_train, holdout_random_y_train,
        eval_set=[(holdout_random_X_train, holdout_random_y_train), (holdout_random_X_val, holdout_random_y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=10),
                   lgb.log_evaluation(period=10, show_stdv=True)])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.082941 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1759
[LightGBM] [Info] Number of data points in the train set: 895057, number of used features: 17
[LightGBM] [Info] Start training from score 58000.483999
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 27029.5	training's l2: 7.30595e+08	valid_1's rmse: 27305.8	valid_1's l2: 7.45608e+08
[20]	training's rmse: 20590.9	training's l2: 4.23986e+08	valid_1's rmse: 20889.8	valid_1's l2: 4.36385e+08
[30]	training's rmse: 17710.6	training's l2: 3.13664e+08	valid_1's rmse: 18041.2	valid_1's l2: 3.25485e+08
[40]	training's rmse: 16163.6	training's l2: 2.61261e+08	valid_1's rmse: 16501.7	valid_1's l2: 2.72308e+08
[50]	training's rmse: 15126.5	training's l2: 2.2881e+08	valid_1's rmse: 15480.8	valid_1's l2: 2.39656e+08
[60]	training's rmse: 14424.6	training's l2: 2.0807e+08

LGBMRegressor(n_estimators=1000)

In [41]:
%%time
X_test = df_test.drop(['target', '계약년월'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = gbm.predict(X_test)


CPU times: user 431 ms, sys: 0 ns, total: 431 ms
Wall time: 52.4 ms


In [42]:
# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)

---

In [24]:
# Holdout - 시간순서
y_train = df_train['target']
X_train = df_train.drop(['target'], axis=1)

# 'date' 기준으로 데이터 정렬
df_train = df_train.sort_values('계약년월')

# 전체 데이터 길이의 20%를 검증 세트 크기로 사용
validation_size = int(len(df_train) * 0.2)

# 훈련 세트와 검증 세트로 분할
train_df = df_train[:-validation_size]
validation_df = df_train[-validation_size:]

holdout_X_train = train_df.drop(['target'], axis=1)
holdout_y_train = train_df['target']

holdout_X_val = validation_df.drop(['target'], axis=1)
holdout_y_val = validation_df['target']




In [25]:
print(f"Train set Date Range :")
display(holdout_X_train['계약년월'].sort_values(ascending=True).head().to_list())
print('...')
display(holdout_X_train['계약년월'].sort_values(ascending=False).head().to_list())

print(f"Valid set Date Range :")
display(holdout_X_val['계약년월'].sort_values(ascending=True).head().to_list())
print('...')
display(holdout_X_val['계약년월'].sort_values(ascending=False).head().to_list())

# 이제 train_df와 validation_df를 사용하여 모델 훈련 및 검증을 진행할 수 있습니다.
del holdout_X_train['계약년월']
del holdout_X_val['계약년월']

Train set Date Range :


[Timestamp('2007-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00'),
 Timestamp('2007-01-01 00:00:00')]

...


[Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00')]

Valid set Date Range :


[Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00'),
 Timestamp('2019-04-01 00:00:00')]

...


[Timestamp('2023-06-01 00:00:00'),
 Timestamp('2023-06-01 00:00:00'),
 Timestamp('2023-06-01 00:00:00'),
 Timestamp('2023-06-01 00:00:00'),
 Timestamp('2023-06-01 00:00:00')]

In [27]:
gbm = lgb.LGBMRegressor(n_estimators=1000)
gbm.fit(holdout_X_train, holdout_y_train,
        eval_set=[(holdout_X_train, holdout_y_train), (holdout_X_val, holdout_y_val)],
        eval_metric='rmse',
        callbacks=[lgb.early_stopping(stopping_rounds=10),
                   lgb.log_evaluation(period=10, show_stdv=True)])

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.031232 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1745
[LightGBM] [Info] Number of data points in the train set: 895058, number of used features: 16
[LightGBM] [Info] Start training from score 49879.244612
Training until validation scores don't improve for 10 rounds
[10]	training's rmse: 20658.3	training's l2: 4.26764e+08	valid_1's rmse: 58851.1	valid_1's l2: 3.46346e+09
[20]	training's rmse: 15733.8	training's l2: 2.47554e+08	valid_1's rmse: 51159.1	valid_1's l2: 2.61725e+09
[30]	training's rmse: 13670.6	training's l2: 1.86886e+08	valid_1's rmse: 46216.5	valid_1's l2: 2.13597e+09
[40]	training's rmse: 12551.4	training's l2: 1.57539e+08	valid_1's rmse: 43187.3	valid_1's l2: 1.86514e+09
[50]	training's rmse: 11796.4	training's l2: 1.39154e+08	valid_1's rmse: 41033.7	valid_1's l2: 1.

LGBMRegressor(n_estimators=1000)

In [28]:
%%time
X_test = df_test.drop(['target', '계약년월'], axis=1)

# Test dataset에 대한 inference를 진행합니다.
real_test_pred = gbm.predict(X_test)

# 앞서 예측한 예측값들을 저장합니다.
preds_df = pd.DataFrame(real_test_pred.astype(int), columns=["target"])
preds_df.to_csv('output.csv', index=False)

CPU times: user 401 ms, sys: 352 µs, total: 402 ms
Wall time: 48.2 ms


- holdout 에서 시간순서로 했을 때 rmse 가 엄청나게 오름